<a href="https://colab.research.google.com/github/LUMII-AILab/NLP_Course/blob/main/notebooks/MSP/TextClassificationWithBERT.ipynb" target="_new"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Teksta klasificēšana ar loģistisko regresiju un BERT

Piezīme: Colab izpildlaika vides parametros izvēlieties bezmaksas GPU (T4).

In [ ]:
!pip install transformers
!pip install datasets
!pip install scikit-learn

## GoEmotions datu kopa

* Publikācija: https://aclanthology.org/2020.acl-main.372/
* Oriģinālā datu kopa: https://github.com/google-research/google-research/tree/master/goemotions
* Priekšapstrādāta **EN** versija: https://huggingface.co/datasets/google-research-datasets/go_emotions
* Priekšapstrādāta **LV** versija: https://huggingface.co/datasets/AiLab-IMCS-UL/go_emotions-lv

## BERT modeļa izvēle un tekstvienību dalītāja ielāde

* Oficiālie Google BERT modeļi - `base` un `large` versijas: https://huggingface.co/google-bert
* Neoficiālas mazākas BERT versijas, piem., `small`: https://huggingface.co/prajjwal1/bert-small
* Latviešu valodai priekšapmācīts BERT modelis: https://huggingface.co/AiLab-IMCS-UL/lvbert
* u.c.

Piezīme: Obligāti jāizmanto modelim atbilstošais tekstvienību dalītājs (*tokenizer*).

In [1]:
from transformers import BertTokenizer

In [ ]:
# Ielādē CPU atmiņā izvēlētā BERT modeļa tekstvienību dalītāju
bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

## Datu kopas ielāde un priekšapstrāde

In [3]:
from datasets import load_dataset

In [4]:
def is_single_label(sample):
    value = sample["labels"]
    if isinstance(value, (list, tuple)):
        return len(value) == 1
    else:
        return False

def to_int_label(sample):
    return {"labels": sample["labels"][0]}

def tokenize(batch):
    return bert_tokenizer(batch["text"], truncation=True)

In [ ]:
data_set = load_dataset("google-research-datasets/go_emotions", "simplified")

filtered_data_set = data_set.filter(is_single_label)
flattened_data_set = filtered_data_set.map(to_int_label)

tokenized_data_set = flattened_data_set.map(tokenize, batched=True)
final_data_set = tokenized_data_set.select_columns(["input_ids", "labels"])

print("data_set:", data_set["train"][0])
print("filtered_data_set:", filtered_data_set["train"][0])
print("flattened_data_set:", flattened_data_set["train"][0])
print("tokenized_data_set:", tokenized_data_set["train"][0])
print("final_data_set:", final_data_set["train"][0])

train_set = final_data_set["train"]
validation_set = final_data_set["validation"]
test_set = final_data_set["test"]

## Loģistiskā regresija

### Izpildes vides sagatavošana

In [6]:
from transformers import BertModel

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import numpy as np
import torch

In [ ]:
# Ielādē izvēlēto BERT modeli GPU atmiņā, inferences režīmā
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Izmantotais procesors:", device)

bert_model = BertModel.from_pretrained("google-bert/bert-base-uncased").to(device).eval()

In [8]:
def get_embeddings(texts, batch_size=256, max_length=512):
    embeddings = []

    for i in range(0, len(texts), batch_size):
        tokenized_batch = bert_tokenizer(
            texts[i:i+batch_size],
            padding = True,
            truncation = True,
            max_length = max_length,
            return_tensors = "pt"
        )

        # Ielādē ieejas vērtības GPU/CPU
        tokenized_batch = {k: v.to(device) for k, v in tokenized_batch.items()}

        with torch.no_grad():
            out = bert_model(**tokenized_batch)
            cls = out.last_hidden_state[:, 0, :]  # [CLS] vektors

        embeddings.append(cls.detach().cpu().numpy())

    return np.vstack(embeddings)

In [9]:
X_train = get_embeddings(flattened_data_set["train"]["text"])
y_train = np.array(flattened_data_set["train"]["labels"])

X_valid = get_embeddings(flattened_data_set["validation"]["text"])
y_valid = np.array(flattened_data_set["validation"]["labels"])

X_test = get_embeddings(flattened_data_set["test"]["text"])
y_test = np.array(flattened_data_set["test"]["labels"])

In [16]:
# Atbrīvo GPU atmiņu (!)
del bert_model
torch.cuda.empty_cache()

In [10]:
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [15]:
print(accuracy_score(y_valid, clf.predict(X_valid)), "- validācijas kopa")
print(accuracy_score(y_test, clf.predict(X_test)), "- testa kopa")

0.5072559366754618 - validācijas kopa
0.5145969498910675 - testa kopa


## BERT pielāgošana

### Izpildes vides sagatavošana

In [17]:
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

### Sagatavošanās bāzes modeļa pielāgošanai

In [ ]:
# Nosaka dažādo klašu skaitu apmācības datu  kopā
label_count = len(data_set["train"].features["labels"].feature.names)
print("label_count", label_count)

# Ielādē RAM izvēlēto BERT modeli, izveido tam atbilstošu klasificēšanas "galvu"
bert_model = BertForSequenceClassification.from_pretrained(
    'google-bert/bert-base-uncased', num_labels=label_count
)

# Nodefinē vienkāršotu novērtēšanas metriku - "accuracy"
def eval_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": float((preds == p.label_ids).mean())}

# Specificē modeļa apmācības hiperparametrus
args = TrainingArguments(
    output_dir = "bert-base-uncased-go_emotions",
    learning_rate = 2e-5,              # tipiski BERT modeļiem
    per_device_train_batch_size = 64,  # atkarībā no GPU atmiņas; var ietekmēt rezultātu
    per_device_eval_batch_size = 128,  # atkarībā no GPU atmiņas
    num_train_epochs = 5,
    fp16 = True,                       # ātrdarbībai uz T4
    metric_for_best_model = "accuracy",
    save_strategy = "epoch",
    eval_strategy = "epoch",
    load_best_model_at_end = True,
    report_to = "none"                 # neizmantot W&B servisu
)

# Izveido apmācības "dzinēju"
trainer = Trainer(
    model = bert_model,
    args = args,
    train_dataset = train_set,
    eval_dataset = validation_set,
    compute_metrics = eval_metrics,
    processing_class = bert_tokenizer,
    data_collator = DataCollatorWithPadding(bert_tokenizer)
)

### Bāzes modeļa pielāgošana klasificēšanas uzdevumam

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.898600,1.410380,0.600923
2,1.324500,1.302983,0.617854
3,1.148800,1.292999,0.617194
4,1.032400,1.312767,0.620053
5,0.929200,1.341430,0.613676


TrainOutput(global_step=2840, training_loss=1.222963467450209, metrics={'train_runtime': 254.0335, 'train_samples_per_second': 714.63, 'train_steps_per_second': 11.18, 'total_flos': 3611501508638112.0, 'train_loss': 1.222963467450209, 'epoch': 5.0})

### Labākās pielāgotās versijas testēšana

In [25]:
trainer.evaluate(test_set)

print("Labākais kontrolpunkts:", trainer.state.best_model_checkpoint)
print("Augstākā precizitāte:", trainer.state.best_metric)

Labākais kontrolpunkts: bert-base-uncased-go_emotions/checkpoint-2272
Augstākā precizitāte: 0.6200527704485488


In [26]:
# Atbrīvo GPU atmiņu (!)
del bert_model
torch.cuda.empty_cache()